In [16]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import os

os.environ["OPENAI_API_KEY"] = "xxx"

In [17]:
# 创建embedding（基于openai）
embeddings = OpenAIEmbeddings()

In [18]:
# 加载文档（具体路径环境不同，不必完全一致）
current_dir = os.getcwd()
loader1 = TextLoader(os.path.join(current_dir, 'dataset/q1.css盒模型.txt'), encoding='utf8')
loader2 = TextLoader(os.path.join(current_dir, 'dataset/q2.基本数据类型.txt'), encoding='utf8')
md_loader = TextLoader(os.path.join(current_dir, 'dataset/q1.前端基础.md'), encoding='utf8')

doc1 = loader1.load()
doc2 = loader2.load()
md_doc = md_loader.load()

In [24]:
# md 文档处理
headers_to_split_on = [
  ("#", "前端基础"),
  ("##", "基础类目"),
  ("###", "具体题目"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_docs = markdown_splitter.split_text(md_doc[0].page_content)
print(md_docs)

[{'content': 'CSS盒模型是指用于布局和渲染HTML元素的一种模型。\n它将每个HTML元素视为一个矩形的盒子，这个盒子包含了元素的内容、内边距、边框和外边距。\nCSS盒模型由以下几个部分组成：\n内容区域（content area）：盒子的实际内容部分，例如文本、图像等。它的大小由元素的width和height属性决定。\n内边距（padding）：内容区域与边框之间的空白区域。可以通过padding属性来设置内边距的大小。\n边框（border）：包围内容区域和内边距的线条。可以通过border属性来设置边框的样式、宽度和颜色。\n外边距（margin）：盒子与周围元素之间的空白区域。可以通过margin属性来设置外边距的大小。\n这些部分相互组合形成一个完整的盒子，它们的大小和样式可以通过CSS来控制和调整。\nCSS盒模型的默认行为是"content-box"，即宽度和高度仅包括内容区域，而不包括内边距、边框和外边距。\n然而，可以通过设置盒模型的box-sizing属性为"border-box"来改变默认行为，使宽度和高度包括内边距和边框。', 'metadata': {'前端基础': '前端基础', '基础类目': 'css', '具体题目': 'css盒模型'}}, {'content': 'JavaScript具有以下基本数据类型：\n布尔类型（Boolean）：表示逻辑上的 true 或 false。\n数字类型（Number）：表示数值，可以是整数或浮点数。\n字符串类型（String）：表示文本数据，用单引号（\'）或双引号（"）括起来。\n空值（Null）：表示一个空值或不存在的值。\n未定义（Undefined）：表示一个未定义的值。\n除了上述基本数据类型，JavaScript 还具有以下复杂数据类型：\n对象类型（Object）：表示复杂的数据结构，可以包含多个键值对。\n数组类型（Array）：表示按顺序排列的一组值，可以通过索引访问。\n函数类型（Function）：表示可执行的代码块。\n日期类型（Date）：表示日期和时间。\n正则表达式类型（RegExp）：表示匹配某种模式的文本。\nSymbol类型（Symbol）：表示唯一的标识符，用于对象属性的键。', 'metadata': {'前端基础': '前端基础

In [ ]:
# 创建字符切割splitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size = 100,
  chunk_overlap  = 20,
  length_function = len,
)

docs = splitter.split_documents(doc1 + doc2)
print(docs)
print(len(docs))

In [ ]:
# 将文档生成向量存储
vectorstore = Chroma.from_documents(docs, embeddings)

# 生成检索器
retriever = vectorstore.as_retriever(search_kwargs={
  "k": 5,
})

In [ ]:
# 查询
interviewer_answers = """
  css盒模型我只知道标准盒模型和怪异盒模型，其他不是很清楚
"""
res = retriever.get_relevant_documents(interviewer_answers)
print(res)